# Corpus Academic

## 1. Lectura de datos!

In [1]:
## abrimos los archivos

import glob

lista_files = glob.glob('academic/*.txt')
corpus = []

for file in lista_files:
    with open(file, 'r', encoding='utf8') as f:
        corpus+=[f.read()]

In [2]:
## número de textos

len(corpus)

8

In [3]:
corpus[0][:250]

"nf0069: when talking about the French gangster film very often and \nparadoxically in French one talks about the policier that's the name of the \ngenre so when you you see the word policier er I-E police which relates to the \npolice it doesn't necessa"

## 2. Preprocesamiento de los datos

In [4]:
## librerias

import spacy
import nltk
import string
from nltk import sent_tokenize
import numpy as np

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pauba\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
## función para remover () y []

def a(test_str):
    ret = ''
    skip1c = 0
    skip2c = 0
    for i in test_str:
        if i == '[':
            skip1c += 1
        elif i == '(':
            skip2c += 1
        elif i == ']' and skip1c > 0:
            skip1c -= 1
        elif i == ')'and skip2c > 0:
            skip2c -= 1
        elif skip1c == 0 and skip2c == 0:
            ret += i
    return ret

In [7]:
## aplicamos la función "a" a cada texto del corpus

corpus = [a(c) for c in corpus]

In [8]:
corpus[0][:250]

"nf0069: when talking about the French gangster film very often and \nparadoxically in French one talks about the policier that's the name of the \ngenre so when you you see the word policier er I-E police which relates to the \npolice it doesn't necessa"

In [9]:
## dividimos por salto de linea cada texto

corpus = [c.replace('♪','').split('\n') for c in corpus]

In [10]:
corpus[0][:5]

['nf0069: when talking about the French gangster film very often and ',
 "paradoxically in French one talks about the policier that's the name of the ",
 'genre so when you you see the word policier er I-E police which relates to the ',
 "police it doesn't necessarily mean that it's to do with a police film or police ",
 "procedure as the you have in the American genres but it's co-, covers all er "]

In [11]:
## eliminamos strings vacíos (cada texto)

corpus = [[sentence.strip() for sentence in c if len(sentence)>0] for c in corpus]

In [12]:
## volvemos a juntar las oraciones :)

sentence_list = [' '.join(c) for c in corpus]

In [13]:
sentence_list[0][:1000]

"nf0069: when talking about the French gangster film very often and paradoxically in French one talks about the policier that's the name of the genre so when you you see the word policier er I-E police which relates to the police it doesn't necessarily mean that it's to do with a police film or police procedure as the you have in the American genres but it's co-, covers all er virtually all the films er to do with crime er pretty much from the nineteen- fifties onwards so er a gangster film and a police film will both be called policier in French and sometimes you'll see the in slang this becomes polar er i'm sorry i hope it's not too awkward if i move er polar is P-O-L-A-R er is yep sf0070: sorry nf0069: okay we're just beginning polar is the slang word for policier so le polar refers to the genre of the police crime gangster movie in French er specifically from the nineteen-fifties late forties fifties onwards and so that i will talk today about a particular genre or subgenre of Fren

In [14]:
## tokens y types

tokens = []

for texto in sentence_list:
    tokens+=texto.split(' ')

tokens = [w.lower() for w in tokens]

In [15]:
## número de tokens y types en el corpus

len(tokens),len(set(tokens))

(67213, 5898)

## análisis!

### 1. Nominalizaciones
El trabajo aquí es con listas de oraciones. Extraemos los sustantivos :)

In [16]:
!pip install spacy
!spacy download en

In [17]:
oraciones_nlp = []

In [18]:
## identificamos el lema y el pos

import spacy
nlp = spacy.load("en_core_web_sm")

for sentence in sentence_list:
    doc = nlp(sentence)
    sent = [(token.lemma_.lower(),token.pos_) for token in doc]
    oraciones_nlp += [sent]

In [19]:
len(oraciones_nlp[0])

8732

In [20]:
def number_tokens_types(oraciones):
    number=[pair[0] for pair in oraciones]
    return len(number),len(set(number)),len(set(number))/len(number)*100

In [21]:
## número de tokens y types para los 8 textos 

for oraciones in oraciones_nlp:
    print(number_tokens_types(oraciones))

(8732, 1159, 13.273018781493356)
(8753, 1022, 11.675996801096765)
(8768, 1235, 14.085310218978103)
(9940, 1154, 11.609657947686117)
(13901, 1713, 12.322854470901374)
(8239, 1128, 13.69098191528098)
(5267, 884, 16.783747864059237)
(7535, 948, 12.581287325812873)


In [22]:
len(tokens)

67213

In [23]:
def number_nouns(oraciones):
    number=[pair for pair in oraciones if pair[1]=='NOUN']
    return len(number),len(set(number)),len(set(number))/len(number)

In [24]:
## número de nouns tokens y types para los 8 textos 

for oraciones in oraciones_nlp:
    print(number_nouns(oraciones))

(1470, 476, 0.3238095238095238)
(1434, 364, 0.25383542538354253)
(1491, 493, 0.3306505700871898)
(1529, 434, 0.2838456507521256)
(2910, 715, 0.24570446735395188)
(1652, 481, 0.29116222760290555)
(1102, 304, 0.27586206896551724)
(1363, 362, 0.26559060895084374)


In [25]:
## la pregunta es: ¿De este número de nouns cuántos son nominalizaciones?

In [26]:
## reglas de nominalization

## reglas de nominalization
## puse las palabras lematizadas, en ese caso, solo es necesario poner las palabras en singular

no_nom = ['thing', 'things', 'somethings', 'something', 'anything', 'everything', 'nothing','original', 'special', 'normal', 'version', 'tutorial', 'moment', 'criminal', 'tradition', 'morning', 'question', 'element', 'quality']
terminacion = ['ment', 'tion', 'sion', 'ibility', 'ity', 'ness', 'al', 'ing'] 

In [27]:
def nominalization(oraciones):
    nom = []
    for word in oraciones:
        if word[0] not in no_nom:
            if word[1]=='NOUN':
                for END in terminacion:
                    if word[0].endswith(END):
                        nom+=[word[0]]
    return nom

In [28]:
nom_list = []

for oraciones in oraciones_nlp:
    nom_list += [nominalization(oraciones)]

In [29]:
## ¿Cuántas de estas son realmente nominalizaciones? Por ejemplo, ¿qué ocurre con "fragment"? Eso podría mejorarse, o al menos
## plantear formas en que podría mirarse de otras formas

nom_list[0]

['hybridity',
 'hybridity',
 'lettering',
 'sexuality',
 'notion',
 'translation',
 'lettering',
 'fiction',
 'material',
 'relation',
 'connection',
 'connection',
 'marketing',
 'revival',
 'revival',
 'mention',
 'movement',
 'action',
 'action',
 'movement',
 'consciousness',
 'adaptation',
 'consciousness',
 'consciousness',
 'consciousness',
 'beginning',
 'expression',
 'highlighting',
 'exception',
 'production',
 'production',
 'production',
 'production',
 'moral',
 'depiction',
 'morality',
 'relation',
 'morality',
 'sexuality',
 'renewal',
 'working',
 'working',
 'popularity',
 'objection',
 'morality',
 'objection',
 'pity',
 'identity',
 'location',
 'understatement',
 'understatement',
 'movement',
 'depiction',
 'activity',
 'reaction',
 'homosexual',
 'bonding',
 'relation',
 'unfolding',
 'action',
 'conversation',
 'action',
 'foreignness',
 'reality',
 'relation',
 'resolution',
 'identity',
 'displacement',
 'sounding',
 'darkness',
 'reversal',
 'discussion',
 '

In [30]:
num_nom_list = []

for oraciones in oraciones_nlp:
    num_nom_list += [len(nominalization(oraciones))/number_nouns(oraciones)[0]*100]

In [31]:
## porcentaje de nominalizaciones con respecto al total de tokens sustantivos para cada texto :)

num_nom_list

[8.16326530612245,
 17.364016736401673,
 15.627095908786048,
 12.49182472204055,
 16.185567010309278,
 20.702179176755447,
 28.85662431941924,
 26.192223037417463]

# 2. Academic Word List

In [32]:
#abriendo la lista de palabras academicas
with open('AcademicWordList.txt') as f:
    AWL=f.read()

In [33]:
#dividir por salto de linea
AWL= AWL.split('\n')

In [34]:
##eliminar espacios en blanco
AWL = [palabra.split(' ') for palabra in AWL if len(palabra)>0]
AWL = [item for sublist in AWL for item in sublist]

In [35]:
AWL.remove('-')

In [36]:
len(AWL)

572

In [37]:
oraciones_nlp[0][:10]

[('nf0069', 'NUM'),
 (':', 'PUNCT'),
 ('when', 'ADV'),
 ('talk', 'VERB'),
 ('about', 'ADP'),
 ('the', 'DET'),
 ('french', 'ADJ'),
 ('gangster', 'NOUN'),
 ('film', 'NOUN'),
 ('very', 'ADV')]

In [38]:
## cada texto tiene la forma

texto = list(zip(*oraciones_nlp[0]))[0]

In [39]:
texto[:10]

('nf0069',
 ':',
 'when',
 'talk',
 'about',
 'the',
 'french',
 'gangster',
 'film',
 'very')

In [40]:
def academic(tokens):
    aca_list=[]
    for word in tokens:
        if word in AWL:
            aca_list+=[word]
    return aca_list

In [41]:
aca_words_percentage = []

aca_words_per_text = []


for oracion in oraciones_nlp:
    texto = list(zip(*oracion))[0]
    aca_words = academic(texto)
    aca_words_per_text+=[aca_words]
    aca_words_percentage += [len(aca_words)/len(texto)*100]

In [42]:
aca_words_percentage

[1.4086120018323407,
 3.2903004684108303,
 3.4329379562043796,
 1.8913480885311873,
 3.1076900942378245,
 4.235950964922928,
 5.942661856844503,
 4.299933642999337]

In [43]:
def number_aca_list(palabras):
    number_aca = []
    for palabra in palabras:
        number_aca+=[pair[0] for pair in palabra]
    return len(number_aca), len(number_aca)/len(tokens)*100

In [44]:
for palabras in aca_words:
    print(number_aca_list(palabras))

(7, 0.010414651927454511)
(7, 0.010414651927454511)
(7, 0.010414651927454511)
(4, 0.005951229672831149)
(12, 0.01785368901849345)
(6, 0.008926844509246724)
(14, 0.020829303854909022)
(8, 0.011902459345662298)
(5, 0.007439037091038936)
(5, 0.007439037091038936)
(7, 0.010414651927454511)
(8, 0.011902459345662298)
(6, 0.008926844509246724)
(7, 0.010414651927454511)
(6, 0.008926844509246724)
(5, 0.007439037091038936)
(4, 0.005951229672831149)
(5, 0.007439037091038936)
(8, 0.011902459345662298)
(7, 0.010414651927454511)
(6, 0.008926844509246724)
(6, 0.008926844509246724)
(6, 0.008926844509246724)
(5, 0.007439037091038936)
(8, 0.011902459345662298)
(11, 0.016365881600285657)
(5, 0.007439037091038936)
(7, 0.010414651927454511)
(8, 0.011902459345662298)
(15, 0.022317111273116808)
(6, 0.008926844509246724)
(9, 0.013390266763870085)
(6, 0.008926844509246724)
(8, 0.011902459345662298)
(7, 0.010414651927454511)
(6, 0.008926844509246724)
(8, 0.011902459345662298)
(8, 0.011902459345662298)
(8, 0.011